# Part 1

## Question 1

In [1]:
# for sample in batch_size:
#     padding_width = input_width + 2*padding
#     padding_height = input_height + 2*padding
#     # Given that output_width and output_height are the same:
#     output_width = output_height = ((input_width - kernel_size + 2*padding)/stride) + 1
#     # Create empty output for this layer
#     output = zeros([total_kernels, output_width, output_height])
#     for layer in input_channels:
#         layer_pad = [sample, layer, :, :]
#         # Add padding
#         for pad in padding:
#             add column of zeros to right to layer_pad
#             add column of zeros to left to layer_pad
#             add column of zeros to top to layer_pad
#             add column of zeros to bottom to layer_pad
#         # Move over layer:
#         for y in range(output_height-1):
#             for x in range(output_width-1):
#                 # If multiple kernels exist, save in different slices of output
#                 for kernel in total_kernels:
#                     patch = layer_pad[y*stride : (y*stride)+kernel_size, x*stride : (x*stride)+kernel_size]
#                     output[kernel, y, x] += sum( patch * kernel[layer, :, :] )
#     sample_output[sample, :, :, :] = output


## Question 2

Question 2. 
For a given input tensor, kernel size, stride and padding (no dilutions) work out
a general function that computes the size of the output.
https://pytorch.org/docs/stable/generated/torch.nn.functional.conv2d.html?highlight=functional%20conv2d#torch.nn.functional.conv2d 

In [2]:
import torch
import numpy as np
import ipykernel
import math
ipykernel.__version__

'6.4.1'

In [3]:
tensor = np.random.rand(1, 2, 27, 27)
weight = np.random.rand(1, 2, 3, 3)

def output_size(input_tensor, kernel_size, stride, padding):
    batch_size, channels, height, width = input_tensor.shape # <- similar to tensor.size() in torch, but '.shape' here since it is numpy
    out_size = (((height - kernel_size + 2*padding) / stride ) + 1)
    return out_size

print(output_size(tensor, 3, 2, 0)) #<- example from the slides: Lecture 3 AlexNet, top right

tensor = torch.rand([1, 2, 27, 27])
weight = torch.rand([1, 2, 3, 3])

print(torch.nn.functional.conv2d(tensor, weight, stride=2,padding=0).size())


13.0
torch.Size([1, 1, 13, 13])


## Question 3

Question 3: 
Write a naive (non-vectorized) implementation of the unfold function in
pseudocode. Include the pseudocode in your report.
https://pytorch.org/docs/stable/generated/torch.nn.functional.unfold.html 

In [4]:
# #input_tensor = [b, c, h, w] #batch_size, channels, height, width

# # Pseudo code naive unfold:
# def naive_unfold(input_tensor, kernel_size, stride, padding):
#     output_size = output_size(input_tensor, kernel_size, stride, padding)
#     #1 extract all patches from the input
#     for sample in b:
#         for channel in c:
#             layer_pad = [sample, layer, :, :]
#             # Add padding
#             for pad in padding:
#                 add column of zeros to right to layer_pad
#                 add column of zeros to left to layer_pad
#                 add column of zeros to top to layer_pad
#                 add column of zeros to bottom to layer_pad
#             n_patches_per_layer = output_size * output_size
#             for y in range(output_size-1):
#                 for x in range(output_size-1):
#                     # x+ y = number of patch, total_patch is 0 first time
#                     total_patch = (x+y) * channel
#                     patch[sample, x+y + total_patch ,:, :] = layer_pad[y*stride : (y*stride)+kernel_size, 
#                                                                         x*stride : (x*stride)+kernel_size]

## Torch Module

In [5]:
import torch
import torch.nn.functional as F
from torch import nn

class Conv2D(nn.Module):
    def __init__(self,in_channels, out_channels, kernel_size=(3,3), stride=1, padding=1):
        super().__init__() # <- belangrijk!
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        
    def forward(self, input_batch):
        batch_size, channels, height, width = input_batch.size()
        print("input_batch = ", input_batch.size())
        
        # output dimensions
        h_out = int(output_size(input_batch, self.kernel_size[0], self.stride, self.padding))
        w_out = int(output_size(input_batch, self.kernel_size[1], self.stride, self.padding))
        
        # unfolded matrix (b, k, p)
        unfolded = F.unfold(input_batch, self.kernel_size, padding=self.padding, stride=self.stride)
        print("unfolded = ", unfolded.size())
        batch_size, k_values_per_patch, patches = unfolded.size()
        
        # reshape to (b, p, k) tensor, than merge b and p to get (b*p,k) tensor
        reshaped = torch.transpose(unfolded, 1, 2).reshape(-1, k_values_per_patch)
        print("X_reshaped = ", reshaped.size())
        
        # Initiate random weights with correct dimensions
        W = torch.rand((k_values_per_patch, self.out_channels)) # - rows: number of nodes in one patch of input. -columns: # of nodes in one pixel in output
        print("W = ", W.size())
        
        # Matrix multiplication to get Y
        Y = torch.mm(reshaped, W) # bmm?
        print("Y = ", Y.size())
        
        # Reshape to get seperate batches back
        Y_reshaped = Y.reshape((batch_size, patches, self.out_channels)) # contains one row-vector for each pixel in output
        print("Y_reshaped = ", Y_reshaped.size())
        
        # Permute to swap axis for p and k
        Y_permuted = torch.permute(Y_reshaped, (0, 2, 1))
        print("Y_permuted = ", Y_permuted.size())
        
        # Fold back to obtain the output of this layer
        output = Y_permuted.reshape(batch_size, self.out_channels, h_out, w_out)
        print("output = ", output.size())
        
        assert output.size() == torch.nn.functional.conv2d(input_batch, W.reshape(self.out_channels, self.in_channels, self.kernel_size[0], self.kernel_size[1]),padding=self.padding).size()
        return output

# We use the Conv2D module by instantiating it, and applying it to an input.
torch.manual_seed(0)
conv = Conv2D(in_channels= 3, out_channels= 8)
input_batch = torch.randn(16, 3, 32, 32)
output_batch = conv(input_batch)

input_batch =  torch.Size([16, 3, 32, 32])
unfolded =  torch.Size([16, 27, 1024])
X_reshaped =  torch.Size([16384, 27])
W =  torch.Size([27, 8])
Y =  torch.Size([16384, 8])
Y_reshaped =  torch.Size([16, 1024, 8])
Y_permuted =  torch.Size([16, 8, 1024])
output =  torch.Size([16, 8, 32, 32])


## Question 4, 5 & 6

In [6]:
class MyConv2DFunc(torch.autograd.Function):
    """
    We can implement our own custom autograd Functions by subclassing
    torch.autograd.Function and implementing the forward and backward
    passes which operate on Tensors.
    """
    @staticmethod
    def forward(ctx, input_batch, kernel, stride=1, padding=1):
        """
        In the forward pass we receive a Tensor containing the input
        and return a Tensor containing the output. ctx is a context
        object that can be used to stash information for backward
        computation. You can cache arbitrary objects for use in the
        backward pass using the ctx.save_for_backward method.
        """
        output_channels = kernel.size()[1]
        
        # your code here
        batch_size, input_channels, height, width = input_batch.size()
        print("X = ", input_batch.size())
        
        kernel_size = int(math.sqrt(kernel.size()[0]/input_channels))
        
        # output dimensions
        h_out = int(output_size(input_batch, kernel_size, stride, padding))
        w_out = int(output_size(input_batch, kernel_size, stride, padding))
        
        # unfolded matrix (b, k, p)
        U = F.unfold(input_batch, (kernel_size, kernel_size), padding=padding, stride=stride)
        print("U = ", U.size())
        batch_size, k_values_per_patch, patches = U.size()
        
        # reshape to (b, p, k) tensor, than merge b and p to get (b*p,k) tensor
        U_reshaped = torch.transpose(U, 1, 2).reshape(-1, k_values_per_patch)
        print("U_reshaped = ", U_reshaped.size())
        
        # Initiate random weights with correct dimensions
        W = torch.rand((k_values_per_patch, output_channels)) # - rows: number of nodes in one patch of input. -columns: # of nodes in one pixel in output
        print("W = ", W.size())
    
        # store objects for the backward
        ctx.save_for_backward(input_batch, U_reshaped, W)
        
        # Matrix multiplication to get Y
        Y = torch.mm(U_reshaped, W) # bmm?
        print("Y = ", Y.size())
        
        # Reshape to get seperate batches back
        Y_reshaped = Y.reshape((batch_size, patches, output_channels)) # contains one row-vector for each pixel in output
        print("Y_reshaped = ", Y_reshaped.size())
        
        # Permute to swap axis for p and k
        Y_permuted = torch.permute(Y_reshaped, (0, 2, 1))
        print("Y_permuted = ", Y_permuted.size())
        
        output_batch = Y_permuted.reshape(batch_size, output_channels, h_out, w_out)
        print("output_batch = ", output_batch.size())
        print("\n")
        
        assert output_batch.size() == torch.nn.functional.conv2d(input_batch, W.reshape(output_channels, input_channels, kernel_size, kernel_size),padding=padding).size()
        return output_batch

    @staticmethod
    def backward(ctx, grad_output):
        """
        In the backward pass we receive a Tensor containing the
        gradient of the loss with respect to the output, and we need
        to compute the gradient of the loss with respect to the
        input
        """
        # retrieve stored objects
        input_batch, X_reshaped, W = ctx.saved_tensors
        # your code here
        
        grad_Y_permuted = grad_output.reshape(grad_output.size()[0], grad_output.size()[1], grad_output.size()[2]*grad_output.size()[2])
        print("grad_Y_permuted = ", grad_Y_permuted.size())
        grad_Y_reshaped = torch.permute(grad_Y_permuted, (0, 2, 1))
        print("grad_Y_reshaped = ", grad_Y_reshaped.size())
        grad_Y = grad_Y_reshaped.reshape(grad_Y_reshaped.size()[0]*grad_Y_reshaped.size()[1],grad_Y_reshaped.size()[2])
        print("grad_Y = ", grad_Y.size())
        kernel_grad = torch.transpose(torch.mm(torch.transpose(grad_Y,0,1), X_reshaped),0,1)
        print("grad_W = ", kernel_grad.size())
        grad_U_reshaped = torch.mm(grad_Y, torch.transpose(W,0,1))
        print("grad_U_reshaped = ", grad_U_reshaped.size())
        grad_U = torch.permute(grad_U_reshaped.reshape(input_batch.size()[0], int(grad_U_reshaped.size()[0]/input_batch.size()[0]), kernel_grad.size()[0]), (0, 2, 1))
        print("grad_U = ", grad_U.size())
        input_batch_grad = F.fold(grad_U, output_size=[input_batch.size()[2], input_batch.size()[3]], kernel_size=(3,3), padding=1)
        print("grad_X = ", input_batch_grad.size())
        return input_batch_grad, kernel_grad, None, None
        
input_channels = 3
output_channels = 8
kernel_size = 3

input_batch = torch.randn(16, 3, 32, 32, requires_grad=True)
kernel = torch.randn(kernel_size*kernel_size*input_channels, output_channels, requires_grad=True)

conv = MyConv2DFunc.apply
output = conv(input_batch, kernel)
loss = output.sum()
loss.backward()

X =  torch.Size([16, 3, 32, 32])
U =  torch.Size([16, 27, 1024])
U_reshaped =  torch.Size([16384, 27])
W =  torch.Size([27, 8])
Y =  torch.Size([16384, 8])
Y_reshaped =  torch.Size([16, 1024, 8])
Y_permuted =  torch.Size([16, 8, 1024])
output_batch =  torch.Size([16, 8, 32, 32])


grad_Y_permuted =  torch.Size([16, 8, 1024])
grad_Y_reshaped =  torch.Size([16, 1024, 8])
grad_Y =  torch.Size([16384, 8])
grad_W =  torch.Size([27, 8])
grad_U_reshaped =  torch.Size([16384, 27])
grad_U =  torch.Size([16, 27, 1024])
grad_X =  torch.Size([16, 3, 32, 32])


# Part 2

## Question 7

In [7]:
import torchvision
from torchvision.transforms import ToTensor
from tqdm import tqdm
import torch

arg = {"data":'./data', "batch": 60000} # with batch = 16 we get a dataloader of length 3750 (*16=60.000)
training_data = torchvision.datasets.MNIST(root=arg['data'], train=True, download=True, transform=ToTensor())
trainloader = torch.utils.data.DataLoader(training_data, batch_size=arg['batch'], shuffle=True, num_workers=2)
test_set = torchvision.datasets.MNIST(root=arg['data'], train=False, download=True, transform=ToTensor())
testloader = torch.utils.data.DataLoader(test_set, batch_size=arg['batch'], shuffle=True, num_workers=2)

In [8]:
for i, data in enumerate(trainloader):
    input, labels = data
    
training = (input[:50000])
training_label = labels[:50000]
validation = input[50000:]
validation_label = labels[50000:]

def loop_over(data, label, step):
    for i in range(0,len(data),step):
        batch = data[i:i+step]
        batch_labels = label[i:i+step]

loop_over(training, training_label, 16)
loop_over(validation, validation_label, 16)

## Question 8
Build this network and tune the hyperparameters until you get a good baseline
performance you are happy with. You should be able to get at least 95% accuracy. If training
takes too long, you can reduce the number of channels in each layer.

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision
import torchvision.transforms
from torchvision.transforms import ToTensor, transforms
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np

In [10]:
class Net(nn.Module):
    def __init__(self, input_chan, kernel, stride, padding, output):
        super().__init__()
        self.conv1 = nn.Sequential(nn.Conv2d(input_chan, 16, kernel, stride, padding), nn.ReLU(), nn.MaxPool2d(2,2))
        self.conv2 = nn.Sequential(nn.Conv2d(16, 32, kernel, stride, padding), nn.ReLU(), nn.MaxPool2d(2,2))
        self.conv3 = nn.Sequential(nn.Conv2d(32, 64, kernel, stride, padding), nn.ReLU(), nn.MaxPool2d(2,2))
        self.out = nn.Linear(64*3*3, output)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = torch.flatten(x, 1)
        output = self.out(x)
        return output

In [11]:
def get_loaders(batch_size):
    loaders = {'train_set' : DataLoader(train_set, 
                                          batch_size=batch_size, 
                                          shuffle=True, 
                                          num_workers=1),
               'val_set' : DataLoader(val_set, 
                                          batch_size=10000, 
                                          shuffle=True, 
                                          num_workers=1),
                'test_set'  : DataLoader(test_set, 
                                          batch_size=len(test_set), 
                                          shuffle=False, 
                                          num_workers=1)}
    return loaders

In [12]:
def validate(net, loaders):
    net.eval()
    correct = 0
    with torch.no_grad():
        for x, y in loaders['val_set']:
            output = net(x)
            _,pred_y = torch.max(output, dim = 1)
            correct += (pred_y == y).float().sum()

    print('accuracy on validation set', (correct / 10000)*100, '%')
    return (correct / 10000)*100

def train(net, loaders, epochs, loss_f, opt):
    train_loss = []
    epoch_list = []
    acc_list = []

    for i in range(epochs):
        print('epoch = ', i)
        for j, (x, y) in enumerate(loaders['train_set']):
            opt.zero_grad()
            x_batch = x
            y_batch = y
            output = net.forward(x_batch)
            loss = loss_f(output, y_batch)
            train_loss.append(loss)
            if j % 1000 == 0:
                print('loss:', loss.item())
            loss.backward()
            opt.step() 
        epoch_list.append(i)
        acc_list.append(validate(net, loaders))
    return train_loss, epoch_list, acc_list

In [13]:
input_chan = 1
output = 10
kernel = 3
stride = 1
padding = 1
lr = 0.0001
net = Net(input_chan, kernel, stride, padding, output)
loss_f = nn.CrossEntropyLoss()
opt = optim.Adam(net.parameters(), lr)


epochs = 5
batch_size = 16
train_set, val_set = torch.utils.data.random_split(training_data, [50000, 10000])
loaders = get_loaders(batch_size) 
train_loss, epoch_list, acc_list = train(net, loaders, epochs, loss_f, opt)

epoch =  0
loss: 2.303809642791748
loss: 0.4514291286468506
loss: 0.27939900755882263
loss: 0.13594754040241241
accuracy on validation set tensor(94.6100) %
epoch =  1
loss: 0.10785152018070221
loss: 0.05007141828536987
loss: 0.25156405568122864
loss: 0.22109884023666382
accuracy on validation set tensor(96.3400) %
epoch =  2
loss: 0.22883853316307068
loss: 0.15962332487106323
loss: 0.08423075079917908
loss: 0.23128293454647064
accuracy on validation set tensor(97.1200) %
epoch =  3
loss: 0.004138742573559284
loss: 0.014569264836609364
loss: 0.051792778074741364
loss: 0.009106600657105446
accuracy on validation set tensor(96.7200) %
epoch =  4
loss: 0.0018573529087007046
loss: 0.053037963807582855
loss: 0.009400795213878155
loss: 0.3510056138038635
accuracy on validation set tensor(97.8300) %


In [14]:
y1 = acc_list
x1 = epoch_list

plt.plot(x1, y1, label = "batch_size = 16" )
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy during training epochs')
plt.xticks(np.arange(0, 20, 1))
plt.savefig('figures/Q8_acc')
plt.show()

## Question 9

In [26]:
augmented = transforms.Compose(
                    [
                    transforms.RandomRotation((-7.0,7.0),fill=(1,)),
                    # transforms.RandomAffine((-3,3),translate =(0.05,0.05)),
                    transforms.ToTensor(),
                    transforms.Normalize((0.1308,), (0.3016,)) 
                    ])

training_data = torchvision.datasets.MNIST(root=arg['data'], train=True, download=True, transform=augmented)
train_set, val_set = torch.utils.data.random_split(training_data, [50000, 10000])    

In [16]:
input_chan = 1
output = 10
kernel = 3
stride = 1
padding = 1
lr = 0.0001
net = Net(input_chan, kernel, stride, padding, output)
loss_f = nn.CrossEntropyLoss()
opt = optim.Adam(net.parameters(), lr)

epochs = 5
batch_size = 16
loaders = get_loaders(batch_size) 
train_loss, epoch_list, acc_list = train(net, loaders, epochs, loss_f, opt)

epoch =  0
loss: 2.2726919651031494
loss: 0.06206151098012924
loss: 0.10813235491514206
loss: 0.06566769629716873
accuracy on validation set tensor(96.0500) %
epoch =  1
loss: 0.060345299541950226
loss: 0.324048787355423
loss: 0.2266087383031845
loss: 0.023933807387948036
accuracy on validation set tensor(96.8300) %
epoch =  2
loss: 0.019474992528557777
loss: 0.09457053989171982
loss: 0.009269594214856625
loss: 0.18167942762374878
accuracy on validation set tensor(97.6100) %
epoch =  3
loss: 0.017100483179092407
loss: 0.0025716719683259726
loss: 0.034040018916130066
loss: 0.0017928642919287086
accuracy on validation set tensor(98.1100) %
epoch =  4
loss: 0.0949849784374237
loss: 0.3918619155883789
loss: 0.021610355004668236
loss: 0.0013072652509436011
accuracy on validation set tensor(98.3600) %


In [17]:
plt.plot(epoch_list, acc_list, label = "batch_size = 16" )
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy during training epochs')
plt.xticks(np.arange(0, 20, 1))
plt.savefig('Q9_acc_augmented')
plt.show()

## Question 10

In [18]:
input_tensor = torch.rand(1, 3, 1024, 768)
conv_layer = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5, stride = 2, padding=1)
print(conv_layer(input_tensor).size())

input_tensor = torch.rand(1, 3, 1920, 1080)
conv_layer = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5, stride = 2, padding=1)
print(conv_layer(input_tensor).size())

input_tensor = torch.rand(1, 8, 1920, 1080)
try:
    conv_layer = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5, stride = 2, padding=1)
    print(conv_layer(input_tensor).size())
except:
    print("ERROR: Dimensions are not correct..")

torch.Size([1, 16, 511, 383])
torch.Size([1, 16, 959, 539])
ERROR: Dimensions are not correct..


## Question 11

In [19]:
b = 16
c = 3
h = 32
w = 32

input_tensor = torch.rand((b, c, h, w))
global_average_pooling = torch.nn.AvgPool2d((h,w), stride=None, padding=0)
global_max_pooling = torch.nn.MaxPool2d((h,w), stride=None, padding=0)
avg_pooled = global_average_pooling(input_tensor)  
max_pooled = global_max_pooling(input_tensor)
print(avg_pooled.size())
print(max_pooled.size())

torch.Size([16, 3, 1, 1])
torch.Size([16, 3, 1, 1])


## Question 12

In [33]:
one_size = transforms.Compose(
                    [
                    transforms.Resize((28,28)),
                    transforms.Grayscale(num_output_channels=1),
                    transforms.ToTensor(),
                    transforms.Normalize((0.1308,), (0.3016,)) 
                    ])

training_data = torchvision.datasets.ImageFolder('mnist-varres/train', transform=one_size)
test_set= torchvision.datasets.ImageFolder('mnist-varres/test', transform=one_size)
train_set, val_set = torch.utils.data.random_split(training_data, [50000, 10000]) 

In [ ]:
trainloader = torch.utils.data.DataLoader(training_data, batch_size=arg['batch'], shuffle=True, num_workers=2)
for i, data in enumerate(trainloader):
    input, labels = data

In [ ]:
input_chan = 1 
output = 10
kernel = 3
stride = 1
padding = 1
lr = 0.0001
net = Net(input_chan, kernel, stride, padding, output)
loss_f = nn.CrossEntropyLoss()
opt = optim.Adam(net.parameters(), lr)

epochs = 5
batch_size = 16
loaders = get_loaders(batch_size) 
train_loss, epoch_list, acc_list = train(net, loaders, epochs, loss_f, opt)

epoch =  0
loss: 2.319960594177246
loss: 1.245522379875183
loss: 0.9869928359985352
loss: 0.8931272625923157
accuracy on validation set tensor(76.1700) %
epoch =  1
loss: 1.0926913022994995
loss: 1.0362154245376587
loss: 0.49100130796432495
loss: 0.41632211208343506
accuracy on validation set tensor(85.2100) %
epoch =  2
loss: 0.39689311385154724
loss: 0.20879657566547394
loss: 0.376911997795105


## Question 13

In Overleaf

## Question 14

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms
from torchvision.transforms import transforms
import matplotlib.pyplot as plt
import torch.nn.functional as F
import numpy as np

class Net(nn.Module):
    def __init__(self, input_chan, kernel, strd, padding, output, pool):
        super().__init__()
        self.pool = pool
        self.conv1 = nn.Sequential(nn.Conv2d(input_chan, 16, kernel, strd, padding), nn.ReLU(), nn.MaxPool2d(2,2))
        self.conv2 = nn.Sequential(nn.Conv2d(16, 32, kernel, strd, padding), nn.ReLU(), nn.MaxPool2d(2,2))
        self.conv3 = nn.Sequential(nn.Conv2d(32, 81, kernel, strd, padding), nn.ReLU(), nn.MaxPool2d(2,2))
        self.out = nn.Linear(81, output)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        if self.pool == 'avg':
            global_avg_pooling = torch.nn.AvgPool2d((x.shape[2],x.shape[3]))
            x = global_avg_pooling(x)
        if self.pool == 'max':
            global_max_pooling = torch.nn.MaxPool2d((x.shape[2],x.shape[3]))
            x = global_max_pooling(x)
        x = torch.flatten(x, 1)
        output = self.out(x)
        return output

In [ ]:
class NetFixed(nn.Module):
    def __init__(self, input_chan, kernel, stride, padding, output):
        super().__init__()
        self.conv1 = nn.Sequential(nn.Conv2d(input_chan, 16, kernel, stride, padding), nn.ReLU(), nn.MaxPool2d(2,2))
        self.conv2 = nn.Sequential(nn.Conv2d(16, 32, kernel, stride, padding), nn.ReLU(), nn.MaxPool2d(2,2))
        self.conv3 = nn.Sequential(nn.Conv2d(32, 64, kernel, stride, padding), nn.ReLU(), nn.MaxPool2d(2,2))
        self.out = nn.Linear(64*3*3, output)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = torch.flatten(x, 1)
        output = self.out(x)
        return output

In [ ]:
def validate(net, loader):
    net.eval()
    correct = 0
    with torch.no_grad():
        for x, y in loader['val_set']: 
            output = net(x)
            _,pred_y = torch.max(output, dim = 1)
            correct = np.array(pred_y == y)  
            accuracy = np.count_nonzero(correct)/len(correct)
        print("Acc = ", accuracy)
    return accuracy

#q14
def loader_loop(net, loader_list, epochs, loss_f, opt):
    avg_loss = []
    epoch_list = []
    acc_list = []
    for i in range(epochs):
        print('epoch = ', i)
        train_loss =[]
        for loader in loader_list:
            for j, (x, y) in enumerate(loader['train_set']):
                opt.zero_grad()
                x_batch = x
                y_batch = y
                output = net.forward(x_batch)
                loss = loss_f(output, y_batch)
                train_loss.append(loss.item())
                loss.backward()
                opt.step()
        print('avg_loss=', sum(train_loss)/len(train_loss))
        avg_loss.append(sum(train_loss)/len(train_loss))
        epoch_list.append(i)
        acc_list.append(validate(net, loader))
    return [avg_loss, epoch_list, acc_list]

def image_folder(path):
    training_data = torchvision.datasets.ImageFolder(path+'mnist-varres/train', transform =  transforms.Compose([transforms.Grayscale(num_output_channels=1), transforms.ToTensor()]))
    test_set= torchvision.datasets.ImageFolder(path+'mnist-varres/test', transform =  transforms.Compose([transforms.Grayscale(num_output_channels=1), transforms.ToTensor()]))
    trainsize = round(0.8*len(training_data))
    train_set, val_set = torch.utils.data.random_split(training_data, [trainsize, len(training_data)-trainsize]) 
    print(len(train_set), len(val_set))
    return [train_set, val_set, test_set]

def get_data(path1, path2, path3):
    data1 = image_folder(path1)
    data2 = image_folder(path2)
    data3 = image_folder(path3)
    return data1, data2, data3

def get_loaders_mixed(batch_size, train_set, val_set, test_set):
    loaders = {'train_set' : DataLoader(train_set, 
                                          batch_size=batch_size, 
                                          shuffle=True, 
                                          num_workers=1),
               'val_set' : DataLoader(val_set, 
                                          batch_size=len(val_set), 
                                          shuffle=True, 
                                          num_workers=1),
                'test_set'  : DataLoader(test_set, 
                                          batch_size=len(test_set), 
                                          shuffle=False, 
                                          num_workers=1)}
    return loaders

## Question15

In [ ]:
def get_parameters(model):
    count=0
    for param in list(model.parameters()):
        n=1
        for s in list(param.size()):
            n = n*s
        count += n
    return count

In [ ]:
input_chan = 1
output = 10
kernel = 3
stride = 1
padding = 1
model_fixed = NetFixed(input_chan, kernel, stride, padding, output)
print('fixed model parameters = ', get_parameters(model_fixed))
model_mixed = Net(input_chan, kernel, stride, padding, output, pool='avg')
print('mixed model parameters = ', get_parameters(model_mixed))

## Question 16

In [ ]:
path1 = '32x32/'
path2 = '48x48/'
path3 = '64x64/'
data32, data48, data64 = get_data(path1, path2, path3)


batch_size = 16
loaders32 = get_loaders_mixed(batch_size, data32[0], data32[1], data32[2])
loaders48 = get_loaders_mixed(batch_size, data48[0], data48[1], data48[2])
loaders64 = get_loaders_mixed(batch_size, data64[0], data64[1], data64[2])
loader_list = [loaders32, loaders48, loaders64]

In [ ]:
input_chan = 1
output = 10
kernel = 3
stride = 1
padding = 1
epochs = 10
lr = 0.0005

net_max = Net(input_chan, kernel, stride, padding, output, pool='max')
loss_f = nn.CrossEntropyLoss()
opt = optim.Adam(net_max.parameters(), lr)
results_max = loader_loop(net_max, loader_list, epochs, loss_f, opt)

net_avg = Net(input_chan, kernel, stride, padding, output, pool='avg')
loss_f = nn.CrossEntropyLoss()
opt = optim.Adam(net_avg.parameters(), lr)
results_avg = loader_loop(net_avg, loader_list, epochs, loss_f, opt)

In [ ]:
y1 = results_avg[2]
x1 = results_avg[1]
plt.plot(x1, y1, label = 'AvgPool')
y2 = results_max[2]
x2 = results_max[1]
plt.plot(x2, y2, label = 'MaxPool')

plt.title('Accuracy of Avg & Max Pooling')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.grid(color='b', linestyle='-', linewidth=0.1)
plt.legend(loc='best')
plt.savefig('figures/q16_acc_avg_max')
plt.show

In [ ]:
y1 = results_avg[0]
x1 = results_avg[1]
plt.plot(x1, y1, label = 'AvgPool')

y2 = results_max[0]
x2 = results_max[1]
plt.plot(x2, y2, label = 'MaxPool')

plt.title('Loss with Global Avg & Max Pooling')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.grid(color='b', linestyle='-', linewidth=0.1)
plt.legend(loc='best')
plt.savefig('figures/q16_Loss_avg_max')
plt.show

## Question17

In [ ]:
def train(net, loaders, epochs, loss_f, opt):
    avg_loss = []
    epoch_list = []
    acc_list = []

    for i in range(epochs):
        print('epoch = ', i)
        train_loss =[]
        for j, (x, y) in enumerate(loaders['train_set']):
            opt.zero_grad()
            x_batch = x
            y_batch = y
            output = net.forward(x_batch)
            loss = loss_f(output, y_batch)
            train_loss.append(loss.item())
            loss.backward()
            opt.step() 
        print('avg_loss=', sum(train_loss)/len(train_loss))
        avg_loss.append(sum(train_loss)/len(train_loss))
        epoch_list.append(i)
        acc_list.append(validate(net, loaders))
    return [avg_loss, epoch_list, acc_list]

In [ ]:
#make loaders of the one size data
def get_loaders_onesize(batch_size, train_set, val_set, test_set):
    loaders = {'train_set' : DataLoader(train_set, 
                                          batch_size=batch_size, 
                                          shuffle=True, 
                                          num_workers=1),
               'val_set' : DataLoader(val_set, 
                                          batch_size=len(val_set), 
                                          shuffle=True, 
                                          num_workers=1),
                'test_set'  : DataLoader(test_set, 
                                          batch_size=len(test_set), 
                                          shuffle=False, 
                                          num_workers=1)}
    return loaders

one_size = transforms.Compose(
                    [
                    transforms.Resize((28,28)),
                    transforms.Grayscale(num_output_channels=1),
                    transforms.ToTensor(),
                    transforms.Normalize((0.1308,), (0.3016,)) 
                    ])

training_data_fz = torchvision.datasets.ImageFolder('mnist-varres/train', transform=one_size)
test_set_fz= torchvision.datasets.ImageFolder('mnist-varres/test', transform=one_size)
train_set_fz, val_set_fz = torch.utils.data.random_split(training_data_fz, [50000, 10000]) 

In [ ]:
#learning rate loops
def lr_loop(input_chan, kernel, stride, padding, output, loader_list, epochs):
    results = []
    lr = 0.0005
    for i in range(10):
        net = Net(input_chan, kernel, stride, padding, output, pool='max')
        loss_f = nn.CrossEntropyLoss()
        opt = optim.Adam(net.parameters(), lr)
        results.append(loader_loop(net, loader_list, epochs, loss_f, opt))
        lr += 0.0005
    return results

def lr_loop_fixed(train_set_fz, val_set_fz, test_set_fz, epochs, batch_size, input_chan, kernel, stride, padding, output):
    results = []
    lr = 0.0005
    for i in range(10):
        net = NetFixed(input_chan, kernel, stride, padding, output,)
        loss_f = nn.CrossEntropyLoss()
        opt = optim.Adam(net.parameters(), lr)
        loaders = get_loaders_onesize(batch_size, train_set_fz, val_set_fz, test_set_fz,)
        results.append(train(net, loaders, epochs, loss_f, opt))
        lr += 0.0005
    return results

In [ ]:
input_chan = 1 
output = 10
kernel = 3
stride = 1
padding = 1
epochs = 5
batch_size = 16

lrResults = lr_loop(input_chan, kernel, stride, padding, output, loader_list, epochs)
lrResultsFixed = lr_loop_fixed(train_set_fz, val_set_fz, test_set_fz, epochs, batch_size, input_chan, kernel, stride, padding, output)
    

In [ ]:
#plot learning rates
def plot_save_lr_results(lr_results, name, title):
    plt.figure(figsize=(8, 6), dpi=80)
    y1 = lr_results[0][2]
    x1 = lr_results[0][1]
    plt.plot(x1, y1, label = '0.0005')

    y1 = lr_results[1][2]
    x1 = lr_results[1][1]
    plt.plot(x1, y1, label = '0.0010')

    y1 = lr_results[2][2]
    x1 = lr_results[2][1]
    plt.plot(x1, y1, label = '0.0015')

    y1 = lr_results[3][2]
    x1 = lr_results[3][1]
    plt.plot(x1, y1, label = '0.0020')

    y1 = lr_results[4][2]
    x1 = lr_results[4][1]
    plt.plot(x1, y1, label = '0.0025')

    y1 = lr_results[5][2]
    x1 = lr_results[5][1]
    plt.plot(x1, y1, label = '0.0030')

    y1 = lr_results[6][2]
    x1 = lr_results[6][1]
    plt.plot(x1, y1, label = '0.0035')

    y1 = lr_results[7][2]
    x1 = lr_results[7][1]
    plt.plot(x1, y1, label = '0.0040')

    y1 = lr_results[8][2]
    x1 = lr_results[8][1]
    plt.plot(x1, y1, label = '0.0045')

    y1 = lr_results[9][2]
    x1 = lr_results[9][1]
    plt.plot(x1, y1, label = '0.0050')

    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.title(title)
    plt.grid(color='b', linestyle='-', linewidth=0.1)
    plt.xticks(np.arange(0, 5, 1))
    plt.legend(loc='best')
    plt.savefig('figures/q17_' + name)
    plt.show

In [ ]:
plot_save_lr_results(lrResults, 'tune_lr_maxpool', 'MaxPool network: Accuracies with learning rates')
plot_save_lr_results(lrResultsFixed, 'tune_lr_maxpool', 'Reshape Network: Accuracies with learning rates')

In [ ]:
def bs_loop(input_chan, kernel, stride, padding, output, superloader, epochs):
    results = []
    lr = 0.0010
    for loader_list in superloader:
        net = Net(input_chan, kernel, stride, padding, output, pool='max')
        loss_f = nn.CrossEntropyLoss()
        opt = optim.Adam(net.parameters(), lr)
        results.append(loader_loop(net, loader_list, epochs, loss_f, opt))
    return results

def bs_loop_fixed(train_set_fz, val_set_fz, test_set_fz, epochs, batch_size, input_chan, kernel, stride, padding, output):
    results = []
    lr = 0.0015
    for batch_size in bs_list:
        net = NetFixed(input_chan, kernel, stride, padding, output,)
        loss_f = nn.CrossEntropyLoss()
        opt = optim.Adam(net.parameters(), lr)
        loaders = get_loaders_onesize(batch_size, train_set_fz, val_set_fz, test_set_fz,)
        results.append(train(net, loaders, epochs, loss_f, opt))
    return results

In [ ]:
def get_loader_list(batch_size, data32, data48, data64):
    loaders32 = get_loaders_mixed(batch_size, data32[0], data32[1], data32[2])
    loaders48 = get_loaders_mixed(batch_size, data48[0], data48[1], data48[2])
    loaders64 = get_loaders_mixed(batch_size, data64[0], data64[1], data64[2])
    loader_list = [loaders32, loaders48, loaders64]
    return loader_list

In [ ]:
input_chan = 1 
output = 10
kernel = 3
stride = 1
padding = 1
epochs = 10
batch_size = 16
bs_list = [4, 16, 32, 64]
superloader = []
for i in bs_list:
    superloader.append(get_loader_list(i, data32, data48, data64))

In [ ]:
bsResultsFixed = bs_loop_fixed(train_set_fz, val_set_fz, test_set_fz, epochs, batch_size, input_chan, kernel, stride, padding, output)
bsResults = bs_loop(input_chan, kernel, stride, padding, output, superloader, epochs)


In [ ]:
def max_value(inputlist):
    return max([sublist[-1] for sublist in inputlist])

In [ ]:
max_acc = max_value(bsResults)
max_acc_fixed = max_value(bsResults)

print("best batch sizes for the networks are:")
print(bs_list.index(max_acc))
print(bs_list.index(max_acc_fixed))

# training with tuned params

In [ ]:
path1 = '32x32/'
path2 = '48x48/'
path3 = '64x64/'
data32, data48, data64 = get_data(path1, path2, path3)


batch_size = 4
loaders32 = get_loaders_mixed(batch_size, data32[0], data32[1], data32[2])
loaders48 = get_loaders_mixed(batch_size, data48[0], data48[1], data48[2])
loaders64 = get_loaders_mixed(batch_size, data64[0], data64[1], data64[2])
loader_list = [loaders32, loaders48, loaders64]

In [ ]:
def test(net, loader):
    net.eval()
    correct = 0
    with torch.no_grad():
        for x, y in loader['test_set']: 
            output = net(x)
            _,pred_y = torch.max(output, dim = 1)
            correct = np.array(pred_y == y)  
            accuracy = np.count_nonzero(correct)/len(correct)
        print("Acc = ", accuracy)
    return accuracy

In [ ]:
input_chan = 1
output = 10
kernel = 3
stride = 1
padding = 1
epochs = 20  #adjust to optimal epochs for final training
lr = 0.0010

netMax = Net(input_chan, kernel, stride, padding, output, pool='max')
loss_f = nn.CrossEntropyLoss()
opt = optim.Adam(netMax.parameters(), lr)
results_max = loader_loop(netMax, loader_list, epochs, loss_f, opt)


In [ ]:
import random
test(netMax, random.choice(loader_list))

In [ ]:
lr = 0.0015
epochs = 20 #adjust to optimal epochs for final training
batch_size = 16
loadersb16 = get_loaders_onesize(batch_size, train_set_fz, val_set_fz, test_set_fz,)
netFixed = NetFixed(input_chan, kernel, stride, padding, output)
loss_f = nn.CrossEntropyLoss()
opt = optim.Adam(netFixed.parameters(), lr)
results_fixed = train(netFixed, loadersb16, epochs, loss_f, opt)

In [ ]:
test(netFixed, loadersb16)

In [ ]:
y1 = results_fixed[2]
x1 = results_fixed[1]
plt.plot(x1, y1, label = 'Reshape Network')

y2 = results_max[2]
x2 = results_max[1]
plt.plot(x2, y2, label = 'MaxPool Network')

plt.title('Accuracy during training epochs')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.xticks(np.arange(0, 21, 1))
plt.grid(color='b', linestyle='-', linewidth=0.1)
plt.legend(loc='best')
plt.savefig('figures/q17_2networks_acc')
plt.show